# Data Parsers
> Let's parse some data.

We can download some data from SiEPIC:

## Imports

In [ ]:
import sax
import requests
import altair as alt
import matplotlib.pyplot as plt

## Lumerical Parser

The [SiEPIC ebeam PDK](https://github.com/SiEPIC/SiEPIC_EBeam_PDK) has a bunch of data files in Lumerical format. Let's download one of them:

In [ ]:
url = "https://raw.githubusercontent.com/SiEPIC/SiEPIC_EBeam_PDK/refs/heads/master/Lumerical_EBeam_CML/EBeam/source_data/ebeam_dc_te1550/dc_gap%3D200nm_Lc%3D0um.sparam"
content = requests.get(url).text
df = sax.parsers.parse_lumerical_dat(content)
df

We see that the parsed dataframe is a dataframe in [tidy format](https://aeturrell.github.io/python4DS/data-tidy.html) with the following columns:

| freq | mag | phase | port_out | port_in | mode_out | mode_in |
|------|-----|-------|----------|---------|----------|---------|

In this case it's a single mode dataframe:

In [ ]:
print(f"{df.mode_in.unique()=}")
print(f"{df.mode_out.unique()=}")

So if we want we can drop those columns:

In [ ]:
df = sax.parsers.parse_lumerical_dat(content)
df = df.drop(columns=["mode_in", "mode_out"])
df

The plotting library [altair](https://github.com/vega/altair) is a perfect fit for visualizing dataframes in tidy format:

In [ ]:
df["wl"] = sax.C_UM_S / df["freq"]
chart = (
    alt.Chart(df.query("port_in=='port_1'"))
    .mark_line()
    .encode(
        x=alt.X("wl", scale=alt.Scale(domain=(df["wl"].min(), df["wl"].max()))),
        y="mag",
        color="port_out",
    )
    .properties(width="container")
)
chart